# **TABELAS STGFILIAL E FILIAL**

## <u>Criando a tabela staging STGFILIAL</u>

- Procedure criada para verificar se a tabela STGFILIAL foi criada no banco. Caso negativo, faremos a criação.
- Além disso essa procedure fará o UPINSERT (MERGE) das informações que virão da tabela PCFILIAL lá no Oracle.

In [2]:
CREATE OR ALTER PROCEDURE SP_INGEST_STGFILIAL
	@CODFILIAL NCHAR(2),
	@EMPRESA NVARCHAR(150),
	@FILIAL NVARCHAR(25)
AS
BEGIN
	----Verifica se a tabela existe
	IF NOT EXISTS (SELECT 1	FROM INFORMATION_SCHEMA.TABLES	WHERE TABLE_NAME = 'STGFILIAL')
    BEGIN
		SET ANSI_NULLS ON

		SET QUOTED_IDENTIFIER ON

		CREATE TABLE [dbo].[STGFILIAL]
		(
			[CODFILIAL] [nchar](2) NOT NULL,
			[EMPRESA] [nvarchar](150) NULL,
			[FILIAL] [nvarchar](25) NULL
		) ON [PRIMARY]

		SET ANSI_PADDING ON

		ALTER TABLE [dbo].[STGFILIAL] ADD  CONSTRAINT [PK_STGFILIAL] PRIMARY KEY CLUSTERED 
		(
			[CODFILIAL] ASC
		)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]

		CREATE UNIQUE NONCLUSTERED INDEX [Index_STGFILIAL_1] ON [dbo].[STGFILIAL]
		(
			[CODFILIAL] ASC
		)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
	END
    
	----Faz o merge das informações
	BEGIN
		IF EXISTS (SELECT 1
		FROM STGFILIAL
		WHERE CODFILIAL = @CODFILIAL)
			BEGIN
			UPDATE STGFILIAL SET EMPRESA = @EMPRESA, FILIAL = @FILIAL WHERE CODFILIAL = @CODFILIAL
		END
    	ELSE
			BEGIN
			INSERT INTO STGFILIAL
				(CODFILIAL, EMPRESA, FILIAL)
			VALUES
				(@CODFILIAL, @EMPRESA, @FILIAL)
		END
	END
END

Commands completed successfully.

Total execution time: 00:00:00.047

In [3]:
SELECT * FROM STGFILIAL

(14 rows affected)

Total execution time: 00:00:00.020

CODFILIAL,EMPRESA,FILIAL
1,JC BROTHERS,LOJA JABAQUARA
10,FFJ CORP,FFJ CORP
11,JC BROTHERS,DISTRIBUICAO ES
12,JC BROTHERS,LOJA JUNDIAI
13,JC BROTHERS,LOJA TRIMAIS
2,JC BROTHERS,DISTRIBUICAO SP
3,JC BROTHERS,ESTOQUE CORPORATIVO
4,JC BROTHERS,ESTOQUE SHOWROOM
5,JFF,JFF CORP
6,BROKER CORP,BROKER CORP


## <u>Realizando as transformações necessárias e criando nova tabela para ser consumida</u>

- Criaremos a coluna `Ordem` para ordernar as filiais uma vez que a PK da tabela staging é do tipo texto. 
- Além disso, preencheremos as informações nulas com informação da JC Brothers

In [4]:
----Analisar as transformações
SELECT CODFILIAL,
    (CASE WHEN CODFILIAL = '99' THEN 'JC BROTHERS' ELSE EMPRESA END) EMPRESA,
    (CASE WHEN CODFILIAL = '99' THEN 'JC BROTHERS' ELSE FILIAL END) FILIAL,
    CAST(CODFILIAL AS smallint) ORDEM
FROM STGFILIAL;

(14 rows affected)

Total execution time: 00:00:00.183

CODFILIAL,EMPRESA,FILIAL,ORDEM
1,JC BROTHERS,LOJA JABAQUARA,1
10,FFJ CORP,FFJ CORP,10
11,JC BROTHERS,DISTRIBUICAO ES,11
12,JC BROTHERS,LOJA JUNDIAI,12
13,JC BROTHERS,LOJA TRIMAIS,13
2,JC BROTHERS,DISTRIBUICAO SP,2
3,JC BROTHERS,ESTOQUE CORPORATIVO,3
4,JC BROTHERS,ESTOQUE SHOWROOM,4
5,JFF,JFF CORP,5
6,BROKER CORP,BROKER CORP,6


In [5]:
CREATE OR ALTER PROCEDURE SP_CARREGA_FILIAL
	@CODFILIAL NCHAR(2),
	@EMPRESA NVARCHAR(150),
	@FILIAL NVARCHAR(25)
AS
BEGIN
	----Verifica se a tabela existe
	IF NOT EXISTS (SELECT 1	FROM INFORMATION_SCHEMA.TABLES	WHERE TABLE_NAME = 'FILIAL')
    BEGIN
		CREATE TABLE [dbo].[Filial] (
    [Cod.Filial] NCHAR (2)      NOT NULL,
    [Empresa]    NVARCHAR (150) NULL,
    [Filial]     NVARCHAR (50)  NULL,
    [Ordem]      SMALLINT       NOT NULL,
    CONSTRAINT [PK_Filial] PRIMARY KEY CLUSTERED ([Cod.Filial] ASC)
);

CREATE NONCLUSTERED INDEX [Index_Filial_2]
    ON [dbo].[Filial]([Empresa] ASC);


CREATE NONCLUSTERED INDEX [Index_Filial_1]
    ON [dbo].[Filial]([Cod.Filial] ASC);


CREATE NONCLUSTERED INDEX [Index_Filial_5]
    ON [dbo].[Filial]([Cod.Filial] ASC, [Empresa] ASC, [Filial] ASC);


CREATE NONCLUSTERED INDEX [Index_Filial_4]
    ON [dbo].[Filial]([Ordem] ASC);


CREATE NONCLUSTERED INDEX [Index_Filial_3]
    ON [dbo].[Filial]([Filial] ASC);
	END
    
	----Faz a transformação das informações
	BEGIN
       DECLARE @ORDEM SMALLINT
       SET @ORDEM = CONVERT(SMALLINT, @CODFILIAL)
	   SET @EMPRESA  = CASE WHEN @EMPRESA IS NULL THEN 'JC BROTHERS' ELSE @EMPRESA END
       SET @FILIAL  =  CASE WHEN @FILIAL IS NULL THEN  'JC BROTHERS' ELSE @FILIAL END
        
    ----faz o merge das informações
        IF EXISTS (SELECT 1 FROM FILIAL WHERE [Cod.Filial] = @CODFILIAL)
			BEGIN
			UPDATE FILIAL SET EMPRESA = @EMPRESA, FILIAL = @FILIAL WHERE [Cod.Filial] = @CODFILIAL
		END
    	ELSE
			BEGIN
			INSERT INTO FILIAL ([Cod.Filial], Empresa, Filial, Ordem)
			VALUES (@CODFILIAL, @EMPRESA, @FILIAL, @ORDEM)
		END
	END
END

Commands completed successfully.

Total execution time: 00:00:00.034

In [2]:
SELECT * FROM FILIAL

(14 rows affected)

Total execution time: 00:00:00.024

Cod.Filial,Empresa,Filial,Ordem
1,JC BROTHERS,LOJA JABAQUARA,1
10,FFJ CORP,FFJ CORP,10
11,JC BROTHERS,DISTRIBUICAO ES,11
12,JC BROTHERS,LOJA JUNDIAI,12
13,JC BROTHERS,LOJA TRIMAIS,13
2,JC BROTHERS,DISTRIBUICAO SP,2
3,JC BROTHERS,ESTOQUE CORPORATIVO,3
4,JC BROTHERS,ESTOQUE SHOWROOM,4
5,JFF,JFF CORP,5
6,BROKER CORP,BROKER CORP,6
